# UMAP plot of the ATAC-seq data

In [1]:
import pandas as pd
import numpy as np
import hicstraw 
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from pandarallel import pandarallel
import cooler
import cooltools
import pybedtools as pbed
pandarallel.initialize()
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns
os.makedirs("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/", exist_ok = True)
pbed.helpers.set_tempdir("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/")
bed_genome_file = "/mnt/iusers01/jw01/mdefscs4/hg38.genome"

plt.rcParams['svg.fonttype'] = 'none'

base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
ATAC_normalised_counts = pd.read_csv(f"{base_dir}/ATAC_seq_analysis/ATAC_DESeq2_quantile_normalized_counts.csv", index_col = 0)
metadata_ATAC = pd.read_csv(f"{base_dir}/metadata/cleaned_ATAC_metadata.csv", index_col=0)

In [4]:
counts_table = ATAC_normalised_counts.iloc[:,3:].copy()
counts_table.index = ATAC_normalised_counts["CHR"] + ":" + ATAC_normalised_counts["START"].astype(str) + "-" + ATAC_normalised_counts["END"].astype(str)

## make some plots for specific genes to assess sample mixups

In [9]:
counts = counts_table.loc["chr12:6789245-6789745"].T
df = pd.DataFrame(counts).merge(metadata_ATAC[["id", "cell_type", 'condition','proper_name']], left_index = True, right_on = "id")
df.columns = ["gene", "id", "cell_type",'condition','proper_name']
df = df.replace(["CD4_SF", "CD8_SF"], ["CD4", "CD8"])
# Create the strip plot
fig = px.strip(df, x="cell_type", y="gene", color="condition",
               hover_name="proper_name",
               color_discrete_map={"Healthy": "blue", "Diseased": "red"},
               title=f'Chromatin accessibility Levels of CD4 promoter Across Cell Types',
               labels={"cell_type": "Cell Type", "gene": "Accessibility Level"})

# Customizing the marker appearance
fig.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    width=600,  # Set the width of the figure in pixels
    height=700  # Set the height of the figure in pixels
)
# Enhancing layout
fig.update_layout(
    title={'text': f'Chromatin accessibility Levels of CD4 promoter Across Cell Types', 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    xaxis_title="Cell Type",
    yaxis_title="Accessibility Level",
    legend_title="Condition",
    font=dict(family="Arial, sans-serif", size=16),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Show the plot
fig.show()
fig.write_image(f"/mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_reviewers_analysis/other_figs/expression_ATAC_CD4.svg")

In [10]:
counts = counts_table.loc["chr2:86790705-86791205"].T
df = pd.DataFrame(counts).merge(metadata_ATAC[["id", "cell_type", 'condition','proper_name']], left_index = True, right_on = "id")
df.columns = ["gene", "id", "cell_type",'condition','proper_name']
df = df.replace(["CD4_SF", "CD8_SF"], ["CD4", "CD8"])
# Create the strip plot
fig = px.strip(df, x="cell_type", y="gene", color="condition",
               hover_name="proper_name",
               color_discrete_map={"Healthy": "blue", "Diseased": "red"},
               title=f'Chromatin accessibility Levels of CD8A promoter Across Cell Types',
               labels={"cell_type": "Cell Type", "gene": "Accessibility Level"})

# Customizing the marker appearance
fig.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    width=600,  # Set the width of the figure in pixels
    height=700  # Set the height of the figure in pixels
)
# Enhancing layout
fig.update_layout(
    title={'text': f'Chromatin accessibility Levels of CD8A promoter Across Cell Types', 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    xaxis_title="Cell Type",
    yaxis_title="Accessibility Level",
    legend_title="Condition",
    font=dict(family="Arial, sans-serif", size=16),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Show the plot
fig.show()
fig.write_image(f"/mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_reviewers_analysis/other_figs/expression_ATAC_CD8A.svg")

In [4]:
import umap
reducer = umap.UMAP(n_components = 2)
latent_rep = reducer.fit_transform(ATAC_normalised_counts.iloc[:,3:].T)

In [10]:
rep = metadata_ATAC.copy()
rep["X"] = latent_rep[:,0]
rep["Y"] = latent_rep[:,1]

In [13]:
rep = rep.sort_values(by="cell_type")
fig = px.scatter(rep, x = "X", y = "Y", color = "cell_type", hover_name = "proper_name", template = "plotly_white", symbol = "female_sex", opacity = 0.8,
height = 600, width = 700)
fig.update_traces(marker={'size': 10})
fig.show()
fig.write_image(f"{base_dir}/ATAC_seq_analysis/figures/ATAC_UMAP_sex.svg")

In [14]:
fig = px.scatter(rep, x = "X", y = "Y", color = "cell_type", hover_name = "proper_name", template = "plotly_white", symbol = "condition", opacity = 0.8,
height = 600, width = 700)
fig.update_traces(marker={'size': 10})
fig.show()
fig.write_image(f"{base_dir}/ATAC_seq_analysis/figures/ATAC_UMAP.svg")